In [1]:
import socketserver
import http.server
import urllib
import base64
import pickle
import http
import socket
import select
import requests
import threading
import os

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
class HTTPServer(socketserver.ThreadingTCPServer):
    
    def __init__(self, addr, request_handler):
        self._thread = None
        self.is_stopped = False
        super().__init__(addr, request_handler)
    
    # from http.server.HTTPServer
    def server_bind(self):
        """Override server_bind to store the server name."""
        socketserver.TCPServer.server_bind(self)
        host, port = self.socket.getsockname()[:2]
        self.server_name = socket.getfqdn(host)
        self.server_port = port
        
    def serve_forever(self, interval=0.05):
        # I attempted to speed up response time by overriding the default 
        # serve_forever with a shorter interval, but it doesn't seem to have worked.
        # oh well. 
        target = super().serve_forever
        self._thread = threading.Thread(target=target, args=(interval,), daemon=True)
        self._thread.start()
        
    def stop(self):
        if self._thread is not None:
            self.shutdown()
            self._thread.join()
            self._thread = None
            self.socket.close()
            self.is_stopped = True
    

In [3]:
import json
getstatic_data = {
    "pressure": {
        "pvUnit": "psi",
        "pvDecimals": 1
    },
    "agitation": {
        "pvUnit": "RPM",
        "manUnit": "%",
        "manName": "Percent Power",
        "pvDecimals": 0,
        "manDecimals": 1
    },
    "mfcs": {
        "co2PvDecimals": 2,
        "airPvDecimals": 2,
        "n2PvDecimals": 2,
        "o2PvDecimals": 2
    },
    "network": {
        "local": False
    },
    "do": {
        "manUpDecimals": 1,
        "pvUnit": "%",
        "manUpUnit": "%",
        "pvDecimals": 1,
        "manUpName": "O2",
        "manDownName": "N2",
        "manDownUnit": "%",
        "manDownDecimals": 1
    },
    "level": {
        "pvUnit": "L",
        "enableLevel": 1,
        "pvDecimals": 2
    },
    "condenser": {
        "pvUnit": "\u00b0C",
        "manUnit": "%",
        "manName": "Filter Oven Heater Duty",
        "pvDecimals": 1,
        "manDecimals": 1
    },
    "ph": {
        "manUpDecimals": 1,
        "pvUnit": "",
        "manUpUnit": "%",
        "pvDecimals": 2,
        "manUpName": "Base",
        "manDownName": "CO2",
        "manDownUnit": "%",
        "manDownDecimals": 1
    },
    "maingas": {
        "pvUnit": "L/min",
        "manUnit": "L/min",
        "manName": "Total Flow",
        "pvDecimals": 2,
        "manDecimals": 2
    },
    "temperature": {
        "pvUnit": "\u00b0C",
        "manUnit": "%",
        "manName": "Main Heater Duty",
        "pvDecimals": 1,
        "manDecimals": 1
    },
    "versions": {
        "server": "V3.0.111",
        "doInputs": 2,
        "database": "V2.4",
        "phInputs": 2,
        "serialNumber": "01961CEE",
        "size": 15,
        "rio": "V3.0.139",
        "temperatureInputs": 0,
        "type": "Mag"
    },
    "pumpa": {
        "speed_control": 1,
        "exists": True,
        "reversible": False,
        "allow_as_base_pump": False
    },
    "pumpb": {
        "speed_control": 1,
        "exists": True,
        "reversible": False,
        "allow_as_base_pump": True
    },
    "pumpc": {
        "speed_control": 1,
        "exists": True,
        "reversible": False,
        "allow_as_base_pump": True
    },
    "pumpsample": {
        "speed_control": 0,
        "exists": True,
        "reversible": True,
        "allow_as_base_pump": False
    },
    "hardware": {
        "testKey1": 0,
        "testKey2": 1,
        "testKey3": 2
    }
}
# speed, exists, reverse, base
pumps3 = [
    (1, True, False, False),
    (1, True, False, True),
    (1, True, False, True),
    (0, True, True, False),
]

pumps15 = [
    (2, True, False, False),
    (1, True, False, True),
    (1, True, False, True),
    (0, True, True, False),
]

pumps80 = [
    (2, True, False, False),
    (2, True, False, False),
    (1, True, False, True),
    (0, True, True, False),
]

pumps = {
    "3L": pumps3,
    "15L": pumps15,
    "80L": pumps80
}

def get_gs(kw):
    return json.dumps(getstatic_data, indent=4)

def set_pump_active(model):
    data = pumps[model]
    order = "pumpa", "pumpb", "pumpc", "pumpsample"
    for cfg, pump in zip(data, order):
        getstatic_data[pump]['speed_control'] = cfg[0]
        getstatic_data[pump]['exists'] = cfg[1]
        getstatic_data[pump]['reversible'] = cfg[2]
        getstatic_data[pump]['allow_as_base_pump'] = cfg[3]
        

In [5]:
def parse_params(params):
    out = {}
    for k, v in params.items():
        if len(v) > 1:
            raise ValueError("Too many arguments for %r: %r" % (k, ", ".join(v)))
        out[k] = v[0]
    return out

def parse_qs(qs):
    return parse_params(urllib.parse.parse_qs(qs))

class HelloProxyError(Exception):
    def __init__(self, code, msg):
        self.response = ProxyResponse(code, msg, {'Connection': 'Close'})
        super().__init__(msg)
        
def get_file_content(fp):
    with open(fp, 'rb') as f:
        return f.read()
        
def content_type_for_file(file):
    ext = os.path.splitext(file)[1]
    if ext == '.json':
        return 'Application/json'
    elif ext == '.css':
        return 'text/css'
    elif ext == '.png':
        return 'image/png'
    elif ext == '.js':
        return 'application/x-javascript'
    elif ext == '.gif':
        return 'image/gif'
    elif ext == '.ico':
        return 'image/icon'
    elif ext == '.html':
        return 'text/html'
    elif ext == '.woff':
        return 'font/woff'
    elif ext == '.ttf':
        return 'application/octet-stream'
    else:
        raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, "Unrecognized MIME type")
        
def byte_encode(msg):
    if isinstance(msg, str):
        return msg.encode('ascii')
    elif isinstance(msg, bytes):
        return msg
    else:
        raise HelloProxyError(http.HTTPStatus.INTERNAL_SERVER_ERROR, "Error encoding response data")
        
class ProxyResponse():
    def __init__(self, code, msg, headers=None):
        self.code = code
        self.msg = byte_encode(msg)
        self.headers = headers or {}
        
def proxy_response_for_file(fullpath):
    msg = get_file_content(fullpath)
    
    headers = {
        'Content-Type': content_type_for_file(fullpath),
        'Content-Length': str(len(msg)),
        'Connection': 'Keep-Alive'
    }
    
    return ProxyResponse(200, msg, headers)

def basic_headers_for_json(j):
    return {
        'Content-Type': 'text/json',
        'Content-Length': str(len(j)),
        'Connection': 'Keep-Alive'
    }
    
class HelloProxyHandler(http.server.SimpleHTTPRequestHandler):
    
    def log_message(self, format, *args):
        pass
    
    def handle(self):
        self.close_connection = False
        while not self.close_connection:
            r, w, l = select.select([self.request], [], [], 0.5)
            if self.server.is_stopped:
                break
            if r:
                self.handle_one_request()
            else:
                break
    
    def do_GET(self):
        res = urllib.parse.urlparse(self.path) 
        kw = parse_qs(res.query)
        call = kw.get('call', None)
        try:
            rsp = self._handle_get_internal(res, kw)
        except HelloProxyError as e:
            rsp = e.response
        except Exception as e:
            msg = "???" + str(e.args[0])
            headers = {
                'Connection': 'Close',
                'Content-Type': 'text/plain',
                'Content-Length': str(len(msg))
            }
            rsp = ProxyResponse(http.HTTPStatus.INTERNAL_SERVER_ERROR, msg, headers)
                  
        if rsp.headers['Connection'].lower() == 'close':
            self.close_connection = True
            
        self.send_response(rsp.code)
        for k, v in rsp.headers.items():
            self.send_header(k, v)
        self.end_headers()
        try:
            self.wfile.write(rsp.msg)
            self.wfile.flush()
        except ConnectionAbortedError:
            self.close_connection = True
        
    def _handle_get_internal(self, res, kw):

        # reject paths that try to navigate out of 
        # the local directory
        if ".." in res.path:
            raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, "Path requests cannot contain access local paths via '..'.")
        
        if res.path.startswith('/webservice'):
            _, ws, handler = res.path.split('/',2)
            if handler == 'interface/':
                return self._handle_server_call(kw)
            else:
                raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, "Unsupported service path: '%s'"%handler)
        fullpath = os.path.join(self.server.basepath, res.path.lstrip("\\/"))
        if os.path.exists(fullpath):
            return proxy_response_for_file(fullpath)
        else:
            raise HelloProxyError(http.HTTPStatus.NOT_FOUND, "Failed to find file '%s'"%res.path)
        
        raise HelloProxyError(http.HTTPStatus.INTERNAL_SERVER_ERROR, "Unreachable code path")
        
    def _handle_fwd_server_call(self, kw):
        url = self.server.proxy_url + self.path
        hfwd = dict(self.headers)
        if self.server.cookies:
            hfwd['Cookie'] = ";".join("%s=%s"%(name, value) for name, value in self.server.cookies.items())
       #print(url)
        fwd = self.server.session.get(url, headers=hfwd, verify=False)
        if fwd.cookies:
            self.server.cookies.update(fwd.cookies)

        msg = fwd.content
        headers = fwd.headers
        headers['Content-Length'] = str(len(msg))
        headers.pop('Transfer-Encoding', None)

        return fwd.status_code, msg, headers
        
        
    USE_DEFAULT = object()
    def _handle_server_call(self, kw):
        
        call = kw.get('call', '').lower()
        func = self.server.get_intercept_handler(call)
        if func:
            try:
                msg = func(kw)
                if not isinstance(msg, str):
                    msg = json.dumps(msg)
                status_code = 200
            except Exception as e:
                status_code = 500
                msg = {"error": str(e)}
                msg = json.dumps(msg)
            headers = basic_headers_for_json(msg)
        else:
            
            # Chrome and requests both don't handle Set-Cookie
            # properly when dealing with localhost (or something like
            # that, whatever something isnt working) so I add 
            # manually based on global server session.
            
            # This has 2 implications: 
            # 1) The client cannot identify themselves across multiple requests
            # 2) The server can only be used for a single client due to needing to store
            #    global data.
            status_code, msg, headers = self._handle_fwd_server_call(kw)
            
        return ProxyResponse(status_code, msg, headers) 
            
    def _server_call_bad_params(self, call, **kw):
        if kw:
            kws = ", ".join("%s=%s" for i in kw.items())
            kws = ": '%s'" % kws
        else:
            kws = ""
        if call is None:
            call = "No call argument provided"
        else:
            call = '%s'%call
            
        msg = "Bad params for server call: %s%s" % (call, kws)
        raise HelloProxyError(http.HTTPStatus.BAD_REQUEST, msg)
        
class HelloProxyServer(HTTPServer):
    def __init__(self, localaddr, remoteaddr, basepath, https=True):
        self.remoteaddr = remoteaddr
        self.basepath = basepath
        if https:
            scheme = "https"
        else:
            scheme = "http"
        self.proxy_url = "%s://%s:%s"%(scheme, remoteaddr[0], remoteaddr[1])
        self.cookies = requests.cookies.RequestsCookieJar()
        self.session = requests.Session()
        self._thread = None
        self.intercepts = {}
        super().__init__(localaddr, HelloProxyHandler)
        
    def get_intercept_handler(self, call):
        return self.intercepts.get(call, None)
    
    def register_intercept_handler(self, call, func):
        self.intercepts[call] = func
        
    def remove_intercept_handler(self, call):
        del self.intercepts[call]
        
    def __del__(self):
        self.socket.close()

In [6]:
serv.stop()
del serv

NameError: name 'serv' is not defined

In [7]:
path = "C:\\Users\\nathan\\Documents\\personal\\test\\uidev"
serv = HelloProxyServer(('localhost', 12347), ('localhost', 8001), path, False)
#serv.register_intercept_handler("getstatic", get_gs)
serv.serve_forever()

In [111]:
# path = "C:\\Users\\nathan\\Documents\\PBS\\sw test\\hello3.0.8"
# serv = HelloProxyServer(('localhost', 12346), ('127.0.0.1', 8001), path)
# serv.register_intercept_handler("getstatic", get_gs)
# serv.serve_forever()

In [98]:
def get_users(kw):
    return """<?xml version="1.0" encoding="windows-1252" standalone="no" ?>
<Reply>
<Result>True</Result>
<Message>User1!@#$%^&*()_,PbStEcH1_2345@pbsbiotech.com</Message>
</Reply>"""

In [99]:
serv.register_intercept_handler("getusers", get_users)

In [100]:
def login(kw):
    val1 = kw['val1']
    val2 = kw['val2']
    print(val1, val2)
    return {"Result": True, "Message":""}

In [101]:
serv.register_intercept_handler("login", login)

In [10]:
def gs_311(kw):
    gs = {
        "network":{
            "local":True
        },
        "versions":{
            "rio":"V3.1.147",
            "server":"V3.1.116",
            "type":"Mag",
            "size":15,
            "database":"V2.4",
            "serialNumber":"01CD188F"
        },
        "agitation":{
            "pvUnit":"RPM",
            "manUnit":"%",
            "manName":"Percent Power",
            "pvDecimals":0,
            "manDecimals":1
        },
        "temperature":{
            "pvUnit":"°C",
            "manUnit":"%",
            "manName":"Main Heater Duty",
            "pvDecimals":1,
            "manDecimals":1,
            "inputs":2
        },
        "do":{
            "pvUnit":"%",
            "manUpUnit":"%",
            "manDownUnit":"%",
            "manUpName":"O2",
            "manDownName":"N2",
            "pvDecimals":1,
            "manUpDecimals":1,
            "manDownDecimals":1,
            "inputs":2
        },
        "ph":{
            "pvUnit":"",
            "manUpUnit":"%",
            "manDownUnit":"%",
            "manUpName":"Base",
            "manDownName":"CO2",
            "pvDecimals":2,
            "manUpDecimals":1,
            "manDownDecimals":1,
            "inputs":2,
            "pHAUseTempComp":0,
            "pHBUseTempComp":1
        },
        "pressure":{
            "pvUnit":"psi",
            "pvDecimals":1
        },
        "level":{
            "pvUnit":"L",
            "pvDecimals":1,
            "enableLevel":1
        },
        "condenser":{
            "pvUnit":"°C",
            "manName":"Filter Oven Heater Duty",
            "manUnit":"%",
            "pvDecimals":1,
            "manDecimals":1
        },
        "maingas":{
            "pvUnit":"L/min",
            "manName":"Total Flow",
            "manUnit":"L/min",
            "pvDecimals":2,
            "manDecimals":2
        },
        "mfcs":{
            "airPvDecimals":2,
            "co2PvDecimals":2,
            "n2PvDecimals":2,
            "o2PvDecimals":2
        }
    }
    
    gs['ph']['inputs'] = 0
    gs['do']['inputs'] = 0
    gs['temperature']['inputs'] = 0
    
    return json.dumps(gs)

serv.register_intercept_handler("getstatic", gs_311)

In [34]:
gbc = """<?xml version="1.0" encoding="windows-1252" standalone="no" ?>
<Reply>
<Result>True</Result>
<Message><Cluster>
<Name></Name>
<NumElts>4</NumElts>
<Cluster>
<Name>CalpHA</Name>
<NumElts>3</NumElts>
<DBL>
<Name>Slope</Name>
<Val>1231293123.87480187416077</Val>
</DBL>
<DBL>
<Name>Offset(%)</Name>
<Val>1.25340652465820</Val>
</DBL>
<DBL>
<Name>Temp(C)</Name>
<Val>25.00000000000000</Val>
</DBL>
</Cluster>
<Cluster>
<Name>CalDOA</Name>
<NumElts>2</NumElts>
<DBL>
<Name>Slope</Name>
<Val>47.77952194213867</Val>
</DBL>
<DBL>
<Name>Offset(%)</Name>
<Val>-95.45653533935547</Val>
</DBL>
</Cluster>
<Cluster>
<Name>CalpHB</Name>
<NumElts>3</NumElts>
<DBL>
<Name>Slope</Name>
<Val>4.12682008743286</Val>
</DBL>
<DBL>
<Name>Offset(%)</Name>
<Val>-4.25458002090454</Val>
</DBL>
<DBL>
<Name>Temp(C)</Name>
<Val>0.00000000000000</Val>
</DBL>
</Cluster>
<Cluster>
<Name>CalDOB</Name>
<NumElts>2</NumElts>
<DBL>
<Name>Slope</Name>
<Val>73.77993011474609</Val>
</DBL>
<DBL>
<Name>Offset(%)</Name>
<Val>-64.69270324707031</Val>
</DBL>
</Cluster>
</Cluster>
</Message>
</Reply>"""

def getbagconfig(kw):
    return gbc

serv.register_intercept_handler("getbagconfig", getbagconfig)